In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import state_union
from nltk.tokenize import  PunktSentenceTokenizer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics as metrics
import scipy


In [2]:
data= pd.read_csv("text_training.csv", delimiter="|").dropna()

In [3]:
print (data.shape)
source = unicode(data["Review Text"][0], 'utf-8')
data= data.reindex(np.random.permutation(data.index))

(47999, 3)


In [4]:
x_train=data["Review Text"]
"""from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x)
X_train_counts.shape"""

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, max_features= None) #, ngram_range=(1,2)
x_data=vectorizer.fit_transform(x_train)
Y_train=data["Category"]



In [5]:
Y_train=data["Category"]
X_tr=x_data[:42000]
Y_tr=Y_train[:42000]
X_test=vectorizer.transform(data["Review Text"][42001:])
Y_test=Y_train[42001:]



In [6]:
def one_hot(labels_train):
    return np.eye(6)[labels_train]



In [7]:
dim=X_tr.shape[1]
w=np.random.normal(0, 0.1, (dim, 16000))
b= np.random.uniform(0, 2*np.pi, 16000)

In [8]:
def phi(X):
     return np.cos(X.dot(w) + b) 

In [9]:
X_tr.shape

(42000, 48597)

In [10]:
Y_tr= Y_tr-1
Y_tr=one_hot(Y_tr.astype(int))
Y_test= Y_test-1
Y_test=one_hot(Y_test.astype(int))
X_test=phi(X_test)
X_tr=phi(X_tr)

In [11]:
def train(X_train, y_train, reg=0):
    xtx = X_train.T.dot(X_train)
    return scipy.linalg.solve(xtx + reg*np.eye(xtx.shape[0]), X_train.T.dot(y_train), sym_pos=True)
    

In [12]:
def predict(model, X):
    ''' From model and data points, output prediction vectors '''
    return np.argmax(X.dot(model), axis=1)



In [13]:
model = train(X_tr,Y_tr, reg=100)

In [14]:
pred_labels_train = predict(model, X_tr)
pred_labels_test = predict(model, X_test)

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In [15]:
pred_labels_train+=1
pred_labels_test+=1
print("Closed form solution")
print("Train accuracy: {0}".format(metrics.accuracy_score(Y_train[:42000], pred_labels_train)))
print("Test accuracy: {0}".format(metrics.accuracy_score(Y_train[42001:], pred_labels_test)))



Closed form solution
Train accuracy: 0.950357142857
Test accuracy: 0.903801267089


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV



In [ ]:
clf=SGDClassifier(penalty="l2", alpha=100)
"""parameters={"alpha":np.linspace(0.0001,50, num=1),
           }
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1, verbose=20)
gs_clf=gs_clf.fit(X_tr, Y_tr)"""

clf.fit(X_tr, Y_tr)

In [ ]:

predict=clf.predict(X_test)

In [ ]:
accuracy= np.mean(predict==Y_test)

In [ ]:
accuracy

from mnist import MNIST
import random
import sklearn.metrics as metrics
import numpy as np
import scipy

NUM_CLASSES = 10
dim = 784
d = 4000
w = np.random.normal(0, 0.1, (dim, d))
b = np.random.uniform(0, 2*np.pi, d)

def load_dataset():
    mndata = MNIST('./data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train = X_train/255.0
    X_test = X_test/255.0
    return (X_train, labels_train), (X_test, labels_test)


def train(X_train, y_train, reg=0):
    ''' Build a model from X_train -> y_train '''
    xtx = X_train.T.dot(X_train)
    return scipy.linalg.solve(xtx + reg*np.eye(xtx.shape[0]), X_train.T.dot(y_train), sym_pos=True)

def train_gd(X_train, y_train, alpha=0.1, reg=0, num_iter=10000):
    ''' Build a model from X_train -> y_train using batch gradient descent '''
    xtx = X_train.T.dot(X_train)
    xty = X_train.T.dot(y_train)
    xtx /= float(X_train.shape[0])
    xty /= float(X_train.shape[0])
    W = np.random.normal(0, 0.3, (X_train.shape[1], NUM_CLASSES))
    for i in range(num_iter):
        gradient = (xtx + reg*np.eye(xtx.shape[0])).dot(W) - xty
        W -= alpha * gradient
    return W

def train_sgd(X_train, y_train, alpha=0.1, reg=0, num_iter=10000):
    ''' Build a model from X_train -> y_train using stochastic gradient descent '''
    W = np.random.normal(0, 0.3, (X_train.shape[1], NUM_CLASSES))
    for i in range(num_iter):
        sample_index = random.randint(0, X_train.shape[0] - 1)
        x_i, y_i = X_train[sample_index], y_train[sample_index]
        xtx = np.outer(x_i, x_i.T)
        xty = np.outer(x_i, y_i.T)
        gradient = (xtx + reg*np.eye(xtx.shape[0])).dot(W) - xty
        W -= alpha * gradient
    return W

def one_hot(labels_train):
    '''Convert categorical labels 0,1,2,....9 to standard basis vectors in R^{10} '''
    return np.eye(NUM_CLASSES)[labels_train]

def predict(model, X):
    ''' From model and data points, output prediction vectors '''
    return np.argmax(X.dot(model), axis=1)

def phi(X):
    ''' Multiply the 784-dimensional MNIST vectors by unit normal '''
    return np.cos(X.dot(w) + b) 


if __name__ == "__main__":
    (X_train, labels_train), (X_test, labels_test) = load_dataset()
    y_train = one_hot(labels_train)
    y_test = one_hot(labels_test)
    X_train, X_test = phi(X_train), phi(X_test)

    model = train(X_train, y_train, reg=0.1)
    pred_labels_train = predict(model, X_train)
    pred_labels_test = predict(model, X_test)
    print("Closed form solution")
    print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))
    print("Test accuracy: {0}".format(metrics.accuracy_score(labels_test, pred_labels_test)))

    model = train_gd(X_train, y_train, alpha=1e-3, reg=0.1, num_iter=20000)
    pred_labels_train = predict(model, X_train)
    pred_labels_test = predict(model, X_test)
    print("Batch gradient descent")
    print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))
    print("Test accuracy: {0}".format(metrics.accuracy_score(labels_test, pred_labels_test)))

    model = train_sgd(X_train, y_train, alpha=1e-3, reg=0.1, num_iter=100000)
    pred_labels_train = predict(model, X_train)
    pred_labels_test = predict(model, X_test)
    print("Stochastic gradient descent")
    print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))
    print("Test accuracy: {0}".format(metrics.accuracy_score(labels_test, pred_labels_test)))

